W tym dokumencie zostały poruszone następujące kwestie: 
1. Podział danych 
2. Przygotowanie danych
3. Analiza ekploracyjna

# Podział danych 
Zdecydowano się na nie przeprowadzanie podziału danych na zbiór testowy i treningowy z pliku artificial_train.data. Podjęto tą decyzję w oparciu na ograniczoną liczbę rekordów, co zwiększało by szanse na ominięcie rekordów niosących istotną informację. Podstawową formą oceny modelu staje się więc mechanizm cross walidacji. Aby zniwelować negatywy płynące z wielkości zbioru danych przyjęto kross walidacje jako sposób oceny modelu

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.base import TransformerMixin

In [ ]:
X_train = pd.read_csv("../Dane/Oryginalne/artificial_train.data", sep=" ", header=None).iloc[:,:-1]
y_train = pd.read_csv("../Dane/Oryginalne/artificial_train_labels.data", sep=' ', header=None)
X_test = pd.read_csv("../Dane/Oryginalne/artificial_test.data", sep=" ", header=None).iloc[:,:-1]

# Przygotowanie danych oraz analiza ekploracyjna
Spojrzenie na dane w ujęciu statystycznym i sprawdzenie znaczących fatków 

In [ ]:
# Sprawdzenie braków danych 
print(X_train.isnull().sum())
print(X_test.isnull().sum())

In [ ]:
# Sprawdzenie rozkładu zmiennej Y 
counterOne = (y_train == 1).sum().sum()
counterMinusOne = (y_train == -1).sum().sum()

print("Liczba wystąpień 1:", counterOne)
print("Liczba wystąpień -1:", counterMinusOne)

In [ ]:
# Sprawdzenie występowania duplikatów  
duplicatedRows = X_train[X_train.duplicated()]
print("Liczba duplikatów:", len(duplicatedRows))

Zauważmy, że dane nie mają braków a rozkład zmiennej odpowiedzi jest idealny w badanym przypadku. Ponadto żaden rekord nie jest zduplikowany.

In [ ]:
# Boxploty dla każdej kolumny w X_train
plt.figure(figsize=(15, 8))
sns.boxplot(data=X_train)
plt.xticks(ticks=range(0, X_train.shape[1], 25), labels=range(0, X_train.shape[1], 25))
plt.title("Boxploty dla kolumn w X_train przed oczyszczeniem nadmiarowych cech")
plt.savefig('../Wyniki/Wykresy/' + "oryginal" + '.jpg', format='jpeg', dpi=300, bbox_inches='tight')
plt.show()

Większość cech ma podobny rozkład z tego samego zakresu wartości. Skłania nas to do wniosku, że dane mogą być silnie skorelowane. 

W związku z tym należy rozważyć przekształcenie zbioru danych pod kątem: 
- odrzucenie rekordów z obserwacjami odstającymi, ponieważ mogą być tylko szumem informacyjnym (ale mogą się również okazać niezwykle inforamtywne)
- redukcja wymiarów poprzez wykrycie kolumn skorelowanych (kolumny skorelowane nie wnoszą kluczowych informacji)

Zauważmy również, że dane składają się jedyne z liczb naturalnych o podobnym rozkładzie co sprawia, że normalizacja i standaryzacja nie ma większego uzasadnienia. Z natury zmiennych nie potrzebne jest kodowanie zmiennych kategorycznych oraz nie możliwe jest tworzenie cech interakcyjnych, ponieważ nie mamy informacji o znaczeniu pragmatycznym zmiennych. 

Należy wziąć pod uwagę, że ratio cech do rekordów wynosi 1:4 co nie jest według nas wystarczająco zadawalające. Dlatego też będziemy dążyć do ograniczenia eksperymentalnie liczby cech, aby uniknąć łatwo możliwego przetrenowania modelu. 

Zbadajmy jeszcze skośność danych

In [ ]:
# Sprawdzenie skośności danych
skewOfX = X_train.skew()
print(skewOfX.min())
print(skewOfX.max())

Wartości od -0.13 do 0.18 sugerują, że dane mają umiarkowanie symetryczny rozkład, więc nie powinno to wpływać na skuteczność działania niektórych modeli uczenia maszynowego. 

Oprócz korzystania z oryginalnego zestawu danych zdecydowano się na następujące przekształcenia zbioru treningowego i testowego:
- PCA - 0.3, 0.4, 0.6, 0.8 - wartość określa ile wariancji w ujęciu całościowym chcemy zachować w danych
- Własnoręczne usuwanie korelacji używając macierzy koleracji przy współczynnikach korelacji - 0.7, 0.075, 0.05
- wykorzystanie cech istotnych dla modelu regresji liniowej 
- wykorzystanie cech istotnych dla modelu random forest 

PCA - Analiza Głównych Składowych


In [ ]:
pca = PCA()
pca.fit(X_train)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
plt.plot(cumulative_variance)
plt.xlabel('Liczba komponentów')
plt.ylabel('Kumulatywna wariancja')
plt.title('Analiza łokcia dla PCA używająć kumulatywnej wariancji')
plt.show()

plt.figure(figsize=(15, 8))
plt.plot(pca.explained_variance_ratio_)
plt.xlabel('Liczba komponentów')
plt.ylabel('Wariancja')
plt.title('Analiza łokcia dla PCA')
plt.savefig('../Wyniki/Wykresy/' + "anlce" + '.jpg', format='jpeg', dpi=300, bbox_inches='tight')
plt.show()

Zauważmy, że wszystkie kolejne cechy wnoszą niewiele do skomulowanej wariancji, a łokieć jest dość wyraźny. 

In [ ]:
def apply_pca_with_mle(X_train, X_test):
    pca = PCA(n_components='mle', random_state=42)
    X_train_transformed = pca.fit_transform(X_train)
    X_test_transformed = pca.transform(X_test)
    return X_train_transformed, X_test_transformed

X_train_MLE, X_test_MLE = apply_pca_with_mle(X_train,X_test)
print(X_train_MLE.shape)
print(X_test_MLE.shape)

Automatyczne redukowane zmiennych nie działa dostatecznie dobrze, bo zostały zredukowane tylko 2 cechy. Dlatego postanowiono pominąć ten sposób przekształcenia. Dodatkowym powodem na ominiecie tej metody bylo dlugość obliczeń.

In [ ]:
def apply_pca_with_var(X_train, n_components, random_state):
    pca = PCA(n_components=n_components, random_state=random_state)
    X_train_transformed = pca.fit_transform(X_train)
    return X_train_transformed

randomState = 42

In [ ]:
X_train_3  = apply_pca_with_var(X_train,0.3,randomState)
print(X_train_3.shape)

In [ ]:
X_train_4 = apply_pca_with_var(X_train,0.4,randomState)
print(X_train_4.shape)

In [ ]:
X_train_6 = apply_pca_with_var(X_train,0.6,randomState)
print(X_train_6.shape)

In [ ]:
X_train_8 = apply_pca_with_var(X_train,0.8,randomState)
print(X_train_8.shape)

Macierz korelacji
Zgodnie z dokumentacją agorytm PCA w pierwszej kolejności odrzuca zmienne wysokoskorelowane więc nie ma uzasadnienia badanie korelacji wynikowych dataframe'ów uzysanych pooprzez stosowanie algorytmu PCA. Zbudujmy dataset ręcznie używając macierzy korelacji.

wartość 0.7 jest standardową wartością z literatury.

In [ ]:
correlation_matrix = X_train.corr()

def remove_correlated_features(X_train, correlation_matrix, threshold):
    correlated_vars = np.where(np.abs(correlation_matrix) > threshold)
    features_to_remove = []
    for var1, var2 in zip(*correlated_vars):
        if var1 != var2 and var1 < var2:
            features_to_remove.append(var2)

    features_to_remove = list(set(features_to_remove))
    print(f'Cechy do usunięcia: {features_to_remove}')
    X_train_filtered = X_train.drop(columns=X_train.columns[features_to_remove], inplace=False)
    print(f'Kształt X_train po usunięciu cech: {X_train_filtered.shape}')
    return X_train_filtered

In [ ]:
X_train_C7 = remove_correlated_features(X_train, correlation_matrix, 0.7)

In [ ]:
X_train_C1 = remove_correlated_features(X_train, correlation_matrix, 0.1)

Zauważmy, że mało jest danych silnie skorelowanych co może pokazywać, że model nie będzie źle dziłać nawet używając całego zbioru danych, bez redukcji liczby kolumn. W celach eksperymentalno - edukacyjnym możemy przetestować co się stanie z modelami gdy zastosujemy zmienne skorelowanie na poziomie 0.075 i 0.05

In [ ]:
X_train_C075 = remove_correlated_features(X_train, correlation_matrix, 0.075)

In [ ]:
X_train_C05 = remove_correlated_features(X_train, correlation_matrix, 0.05)

Pozyskiwanie najważniejszych cech z perspektywy algorytmu LASSO

Została użyta regresja LASSO, ponieważ wprowadza regularyzację L1, co skutkuje rzadkimi wagami cech.

In [ ]:
lasso_cv_model = LassoCV(alphas=[0.01, 0.1, 1.0, 10.0], cv=5, random_state=42)
lasso_cv_model.fit(X_train, y_train)

sel = SelectFromModel(lasso_cv_model, prefit=True)

sel.fit(X_train, y_train.values.reshape(-1),random_state=42)

selected_feat = X_train.columns[(sel.get_support())]

X_train_LA = X_train.loc[:,selected_feat]
X_test_LA = X_test.loc[:,selected_feat]

print(X_train_LA.shape)

Pozyskiwanie najważniejszych cech z perspektywy algorytmu Random Forest

Został użyty Random Forest, ze względu na jego zdolność do oceny ważności cech na podstawie kryterium Gini lub Entropii. 

In [ ]:
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
}

rf_model = RandomForestClassifier()
grid_search = GridSearchCV(rf_model, param_grid, cv=5)
grid_search.fit(X_train, y_train.values.flatten())

best_params = grid_search.best_params_

sel = SelectFromModel(grid_search.best_estimator_)

sel.fit(X_train, y_train.values.ravel())

selected_feat = X_train.columns[(sel.get_support())]
len(selected_feat)

X_train_RF = X_train.loc[:,selected_feat]
X_test_RF = X_test.loc[:,selected_feat]
print(X_test_RF.shape)

Zauważono, że użycie powyższych algorytmu PCA wykluczyło cechy, które zawierały obserwacje odstające. Jedynie gdzie widzimy nadal obserwacje odstające to dla ręcznym usuwaniu cech na podstawie macierzy korelacji dla współczynnika 0.7. Ale uważamy, że będzie to dobry miernik istoty tych zmiennych na wyniki modelu. 

In [ ]:
plt.figure(figsize=(15, 8))
sns.boxplot(data=X_train_8)
plt.xticks(ticks=range(0, X_train_8.shape[1], 5), labels=range(0, X_train_8.shape[1], 5))
plt.title("Boxploty dla kolumn w X_train po oczyszczeniu cech metodą PCA dla współczynnika 0.8")
plt.savefig('../Wyniki/Wykresy/' + "PCA08" + '.jpg', format='jpeg', dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(15, 8))
sns.boxplot(data=X_train_C075)
plt.xticks(ticks=range(0, X_train_C075.shape[1], 10), labels=range(0, X_train_C075.shape[1], 10))
plt.title("Boxploty dla kolumn w X_train po oczyszczeniu cech używając macieży korelacji i współczynnika 0.7")
plt.savefig('../Wyniki/Wykresy/' + "CORR07" + '.jpg', format='jpeg', dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(15, 8))
sns.boxplot(data=X_train_RF)
plt.xticks(ticks=range(0, X_train_RF.shape[1], 4), labels=range(0, X_train_RF.shape[1], 4))
plt.title("Boxploty dla kolumn w X_train po oczyszczeniu cech używając algorytmu RandomForest")
plt.savefig('../Wyniki/Wykresy/' + "RandomForest" + '.jpg', format='jpeg', dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(15, 8))
sns.boxplot(data=X_train_LA)
plt.xticks(ticks=range(0, X_train_LA.shape[1], 1), labels=range(0, X_train_LA.shape[1], 1))
plt.title("Boxploty dla kolumn w X_train po oczyszczeniu cech używając algorytmu LASSO")
plt.savefig('../Wyniki/Wykresy/' + "LASSO" + '.jpg', format='jpeg', dpi=300, bbox_inches='tight')
plt.show()
